# ControlVideo

### Installation

Only for colab

In [ ]:
!git clone --recurse-submodules https://github.com/rossiyareich/marching-waifu-x.git
%cd marching-waifu-x

Only for local install (run `setup.sh` or `setup.bat` and `huggingface-cli login` beforehand!)

In [ ]:
%cd ../

Project setup

In [ ]:
!nvidia-smi

# Install requirements
!python -m pip install -r requirements.txt
!python -m pip install -r ext/Real-ESRGAN/requirements.txt

# Create directories
!mkdir data/dataset/nerf/original/
!mkdir data/dataset/nerf/train/

# Install local packages
%cd ext/Real-ESRGAN/
!python setup.py develop
%cd ../../

In [ ]:
%cd scripts

In [ ]:
import sys
sys.path.append("..")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### Configuration

In [ ]:
controlvideo = '''
--out_path "../data/dataset/nerf/original/"
--sd_repo "rossiyareich/Nabylon-v1.0-fp16"
--vae_repo "stabilityai/sd-vae-ft-mse"
--controlnet_repo "lllyasviel/control_v11p_sd15_openpose"
--ifnet_path "../data/checkpoints/flownet.pkl"
--cache_dir "../data/checkpoints/"
--prompt "(masterpiece, best quality)+, 1girl, white hoodie, earmuffs, leggings, white scarf, black gloves, white socks, short blue hair, blue eyes, bangs"
--negative_prompt "EasyNegative, (worst quality, low quality, logo, text, watermark, username, nsfw), inaccurate hands and fingers"
--textual_inversion_path "../data/embeddings/"
--controlnet_conditioning_path "../data/dataset/conditioning/" 
--video_length 16
--num_inference_steps 50
--guidance_scale 10.0
--smoother_steps "19,20"
--window_size 4
--controlnet_conditioning_scale 1.0
'''.replace('\n', ' ')

realesrgan = '''
--in_path "../data/dataset/nerf/original/"
--out_path "../data/dataset/nerf/train/"
--outscale 4.0
--tile_pad 192
--pre_pad 16
--face_enhance True
--fp32 False
--gpu_id 0
'''.replace('\n', ' ')

### Inference

In [ ]:
import glob
import os
import pathlib
import PIL.Image
from src.utils.image_wrapper import *

def load_images():
    video_frames = []
    controlnet_conditions = []
    for filepath in sorted(glob.glob("../data/dataset/conditioning/*.png")):
        pl = pathlib.Path(filepath)
        video_frame_path = os.path.join("../data/dataset/nerf/original/", f"{pl.stem}.png")
        video_frames.append(PIL.Image.open(video_frame_path))
        controlnet_conditions.append(PIL.Image.open(filepath))
    return (video_frames, controlnet_conditions)

def display_at_index(index, video_frames, controlnet_conditions):
    display(image_wrapper(video_frames[index], "pil").concatenate(
        image_wrapper(controlnet_conditions[index], "pil")
    ).to_pil())

In [ ]:
!python inference_controlvideo.py {controlvideo}

In [ ]:
import time
from IPython.display import clear_output

display_interval = 1

video_frames, controlnet_conditions = load_images()
length = len(video_frames)

for i in range(length):
    display_at_index(i, video_frames, controlnet_conditions)
    time.sleep(display_interval)
    if i < length - 1:
        clear_output(True)

In [ ]:
!python inference_realesrgan.py {realesrgan}